In [43]:
import os
from dotenv import load_dotenv

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration, OrchestrationHandoffs, RoundRobinGroupChatManager
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.agents.runtime import InProcessRuntime

In [46]:
# Here, we connect to the Azure AI Foundry, and get the necessary environment variables
# including the project endpoint, model deployment name, and tenant ID.

load_dotenv()
project_endpoint = os.getenv("PROJECT_ENDPOINT")
endpoint = os.getenv("ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
api_key = os.getenv("API_KEY")
api_version = os.getenv("API_VERSION")

In [47]:
# This block of code connects to the MCP Server
async with MCPStdioPlugin(
    name="InventoryManagementMCPPlugin",
    description="Inventory Management MCP Plugin",
    command="python",
    args=["server.py"],
) as inventory_plugin:
    pass
 
chat_service = AzureChatCompletion(
    endpoint=endpoint,
    deployment_name=model_deployment,
    api_version=api_version,
    api_key=api_key,
)

In [48]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    if message.content:
        print(f"🤖{message.name}\n{message.content}")

In [49]:
# Create the agent
agent = ChatCompletionAgent(
    name="inventory-agent",
    description="An agent that manages inventory levels and provides recommendations for restocking or clearance.",
    service=chat_service,
    plugins=[inventory_plugin],
    instructions="""
    You are an expert inventory management assistant for retail operations. Analyze product data and provide actionable inventory recommendations based on these specific criteria:

    - RESTOCK RECOMMENDATIONS: Identify products where inventory < 10 units AND weekly sales > 15 units. These products are at risk of stockout and need immediate replenishment.
    - CLEARANCE RECOMMENDATIONS: Identify products where inventory > 20 units AND weekly sales < 5 units. These products have slow turnover and should be considered for clearance pricing.
    - BALANCED INVENTORY: Products not meeting either criteria above are considered to have balanced inventory levels.

    When reporting:
    1. Format your responses in clear, organized sections with appropriate headings
    2. Include relevant metrics for each product (current inventory, weekly sales, days of supply)
    3. Sort recommendations by priority (highest sales-to-inventory ratio for restocking, lowest for clearance)
    4. Provide specific, concise recommendations without asking follow-up questions
    5. When providing inventory metrics, use actual numbers from the data rather than general statements

    Your goal is to help optimize inventory levels, minimize holding costs, and prevent stockouts through data-driven recommendations.
    """
)

await inventory_plugin.connect()

In [ ]:
user_input = "Can you tell me what the current inventory levels are for all products?"
print(f"# 😃: {user_input}")
response = await agent.get_response(messages=user_input)

print(f"# {response.name}: {response}")

# 😃: Can you tell me what the current inventory levels are for all products?
# inventory-agent: Here are the current inventory levels for all products:

- Chai: 39 units
- Chang: 0 units
- Aniseed Syrup: 13 units
- Chef Anton's Cajun Seasoning: 53 units
- Chef Anton's Gumbo Mix: 25 units
- Grandma's Boysenberry Spread: 120 units
- Uncle Bob's Organic Dried Pears: 100 units
- Northwoods Cranberry Sauce: 20 units
- Mishi Kobe Niku: 29 units
- Ikura: 31 units
- Queso Cabrales: 22 units
- Queso Manchego La Pastora: 86 units
- Konbu: 0 units
- Tofu: 35 units
- Genen Shouyu: 39 units
- Pavlova: 29 units
- Alice Mutton: 0 units
- Carnarvon Tigers: 42 units
- Teatime Chocolate Biscuits: 125 units
- Sir Rodney's Marmalade: 40 units
- Casey's Canned Peaches: 65 units

Let me know if you want me to analyze these inventory levels with weekly sales data to provide restocking or clearance recommendations.


In [58]:
user_input = "Can you tell me what the current inventory levels are for all products?"
print(f"😃: {user_input}")
response = await agent.get_response(messages=user_input)

print(f"🤖: {response.name}: {response}")

😃: Can you tell me what the current inventory levels are for all products?
🤖: inventory-agent: Current Inventory Levels for All Products:

- Chai: 39 units
- Chang: 0 units
- Aniseed Syrup: 13 units
- Chef Anton's Cajun Seasoning: 53 units
- Chef Anton's Gumbo Mix: 25 units
- Grandma's Boysenberry Spread: 120 units
- Uncle Bob's Organic Dried Pears: 100 units
- Northwoods Cranberry Sauce: 20 units
- Mishi Kobe Niku: 29 units
- Ikura: 31 units
- Queso Cabrales: 22 units
- Queso Manchego La Pastora: 86 units
- Konbu: 0 units
- Tofu: 35 units
- Genen Shouyu: 39 units
- Pavlova: 29 units
- Alice Mutton: 0 units
- Carnarvon Tigers: 42 units
- Teatime Chocolate Biscuits: 125 units
- Sir Rodney's Marmalade: 40 units
- Casey's Canned Peaches: 65 units

If you want, I can analyze these inventory levels with corresponding sales data for recommendations on restocking, clearance, or balanced inventory.


In [ ]:
user_input = "Which products in the inventory would you recommend for clearance?"
print(f"😃: {user_input}")
response = await agent.get_response(messages=user_input)

print(f"🤖: {response.name}: {response}")

# 😃: Which products in the inventory would you recommend for clearance?
# inventory-agent: Clearance Recommendations:

The following products have inventory levels greater than 20 units and weekly sales below 5 units, indicating slow turnover and potential for clearance pricing:

1. Sir Rodney's Marmalade
   - Current Inventory: 40 units
   - Weekly Sales: 2 units
   - Days of Supply: 20 days (approximate based on sales)

Recommendation: Consider applying clearance pricing or promotional discounts to Sir Rodney's Marmalade to improve turnover and reduce holding costs.


In [ ]:
user_input = "What are the best sellers in the Inventory this week?"
print(f"😃: {user_input}")
response = await agent.get_response(messages=user_input)

print(f"🤖: {response.name}: {response}")

# 😃: What are the best sellers in the Inventory this week?
# inventory-agent: Here are the best-selling products in the inventory this week, sorted by units sold:

1. Queso Cabrales - 91 units sold
2. Teatime Chocolate Biscuits - 81 units sold
3. Chef Anton's Cajun Seasoning - 67 units sold
4. Queso Manchego La Pastora - 62 units sold
5. Casey's Canned Peaches - 50 units sold
6. Chai - 45 units sold
7. Pavlova - 45 units sold
8. Genen Shouyu - 33 units sold
9. Aniseed Syrup - 32 units sold

These products have the highest sales volume this week and are the current best sellers. Let me know if you want inventory or restock recommendations for any of these.


In [ ]:
user_input = "I forget what items in the Inventory are out of stock, can you tell me what they are?"
print(f"😃: {user_input}")
response = await agent.get_response(messages=user_input)

print(f"🤖: {response.name}: {response}")

# 😃: I forget what items in the Inventory are out of stock, can you tell me what they are?
# inventory-agent: The following items in the inventory are currently out of stock (0 units available):

- Chang
- Konbu
- Alice Mutton

These products have no inventory at the moment.


In [ ]:
user_input = "Can you increase the stock amount of Teatime Chocolate Biscuits to 125?"
print(f"😃: {user_input}")
response = await agent.get_response(messages=user_input)

print(f"🤖: {response.name}: {response}")

# 😃: Can you increase the stock amount of Teatime Chocolate Biscuits to 125?
# inventory-agent: The stock amount for Teatime Chocolate Biscuits is already at 125 units, so no change was needed. If you need further adjustments or assistance, please let me know!


In [ ]:
user_input = "Can you write a marketing campaign for the products in our inventory that are good candidates for clearance? Do not include inventory that is discontinued. Give the marketing campaign a sense of urgency and filled with clickbait."
print(f"😃: {user_input}")
response = await agent.get_response(messages=user_input)

print(f"🤖: {response.name}: {response}")

# 😃: Can you write a marketing campaign for the products in our inventory that are good candidates for clearance? Do not include inventory that is discontinued. Give the marketing campaign a sense of urgency and filled with clickbait.
# inventory-agent: Here is a high-impact marketing campaign draft targeting products in our inventory suitable for clearance, creating urgency and clickbait appeal. The campaign excludes any discontinued products:

---

**🔥 Flash Clearance Blowout! Unbelievable Deals Ending Soon! 🔥**

Hurry, don’t miss your chance to snag these HOT clearance steals before they vanish! Our shelves are packed with popular products moving slow — and we’re passing MASSIVE discounts directly to you. Act fast, quantities won’t last!

**Clearance Hotlist (Oversupply Meets Low Sales):**

- Grandma's Boysenberry Spread  
  Inventory: 120 units | Weekly Sales: 23 units | Days of Supply: 5.22  
  Taste the sweetness before it’s gone! Limited-time markdown!

- Uncle Bob’s Organic Dri